In [1]:
import src.data.patient_data as patient_data
import numpy as np
import src.data.measurements_data as measurements_data
import pandas as pd
from dateutil.relativedelta import relativedelta

In [2]:
df_patient = patient_data.load()
df_patient.drop(columns=["DOB", "ID", "Study Date", "Sex"]).agg(
    ["count", "mean", "min", "max"]
).style.set_precision(1)


** Loading patient data **


/Applications/anaconda3/envs/phd/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/Users/tristan.trebaol/Desktop/PhD/Code/phd/src/data/patient_data.py:187: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Height.loc[df.ID == "60"] = tmp * 100



* Dropping unnecessary columns from patient data *
Columns filtered: ['ID', 'Study Date', 'DOB', 'Age', 'Sex', 'Height', 'Weight', 'Predicted FEV1', 'FEV1 Set As']
Columns dropped: {'Unable Informed Consent', 'Comments', 'Inconvenience Payment', 'Pulmonary Exacerbation', 'Unable Sputum Samples', 'Study Number', 'Age 18 Years', 'Genetic Testing', 'Study Email', 'Informed Consent', 'Date Last PE Stop', 'Telemetric Measures', 'Sputum Samples', 'Less Exacerbation', 'GP Letter Sent', 'Freezer Required', 'Date Last PE Start', 'Remote Monitoring App User ID', 'Transplant Recipients', 'CFQR Quest Comp', 'Date Consent Obtained', 'Hospital'}

* Correcting patient data *
ID 60: Corrected height 60 from 1.63 to 163.0
ID 66: Corrected height for ID 66 from 1.62 to 162.0
Replace Age by calculate age
Drop FEV1 Set As and Predicted FEV1
Compute Calculated Predicted FEV1

* Applying data sanity checks *
Loaded patient data with 147 entries (147 initially)


/var/folders/zq/v2r6yn111s3gpdf8lzf72xvw0000gn/T/ipykernel_47896/1916377473.py:4: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  ).style.set_precision(1)


,Age,Height,Weight,Predicted FEV1
count,147.0,147.0,147.0,147.0
mean,31.5,166.3,63.0,3.4
min,18.0,143.0,34.3,2.1
max,66.0,189.0,117.3,4.7


In [5]:
df_patient[df_patient.ID == "194"]

,ID,Study Date,DOB,Age,Sex,Height,Weight,Predicted FEV1
104,194,2016-07-22,1983-05-20,33,Male,165.0,54.5,3.648


In [3]:
df_measurements = measurements_data.load()
df_measurements.groupby(["ID"]).count()


** Loading measurements data **

* Dropping unnecessary columns from measurements data *
Columns filtered ['User ID', 'UserName', 'Recording Type', 'Date/Time recorded', 'FEV 1', 'Weight in Kg', 'O2 Saturation', 'Pulse (BPM)', 'Rating', 'Temp (deg C)']
Dropping columns {'Sputum sample taken?', 'FEV 10', 'Activity - Points', 'Activity - Steps', 'Predicted FEV', 'FEV 1 %', 'Calories'}

* Renaming columns *
Renamed columns {'Date/Time recorded': 'Date Recorded', 'FEV 1': 'FEV1', 'Weight in Kg': 'Weight (kg)'}


/Users/tristan.trebaol/Desktop/PhD/Code/phd/src/data/measurements_data.py:101: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(datadir + "mydata.csv")



* Applying data sanity checks *

FEV1
Dropping 1 entries with FEV1 = 3.45 for user Kings004

Weight (kg)
Dropping 2 entries with Weight (kg) = 6.0 for user Papworth033
Dropping 1 entries with Weight (kg) = 0.55 for user Kings013
Dropping 1 entries with Weight (kg) = 8.262500000000001 for user Papworth017
Dropping 1 entries with Weight (kg) = 1056.0 for user leeds01730
Dropping 1 entries with Weight (kg) = 20.0 for user Papworth019
Warning - ID PapworthSummer has Weight (28.9375) outside 30-122 kg range
Warning - ID PapworthSummer has Weight (29.200000000000003) outside 30-122 kg range
Warning - ID EmemTest has Weight (14.9625) outside 30-122 kg range
Warning - ID FPH0011 has Weight (7.8) outside 30-122 kg range

Pulse (BPM)
Dropping 14 entries with Pulse (BPM) == 511)
       Pulse (BPM)      UserName
60638        511.0   Papworth002
60989        511.0   Papworth001
61026        511.0    leeds01050
61374        511.0    leeds01320
63126        511.0      Kings005
63525        511.0    

,Recording Type,Date Recorded,FEV1,Weight (kg),O2 Saturation,Pulse (BPM),Rating,Temp (deg C),Patient_ID
ID,,,,,,,,,
100,542,542,19,59,60,60,178,0,542
101,944,944,77,103,104,107,319,0,944
102,1797,1797,162,182,183,183,549,180,1797
107,445,445,14,42,44,42,131,15,445
113,1185,1185,14,118,131,128,391,1,1185
...,...,...,...,...,...,...,...,...,...
82,756,756,43,75,75,73,230,78,756
88,20,20,3,3,3,3,6,0,20
92,637,637,35,57,71,69,217,11,637


In [4]:
# Merge patient and measurements data

# 195 recordings removed after merging with id mapping file
# 0 recordings removed after merging with patient data
df = df_measurements.merge(df_patient, on="ID", how="inner")
df = df.sort_values(by="ID").drop(
    columns=[
        "Recording Type",
        "Study Date",
        "DOB",
        "Age",
        "Sex",
        "Height",
        "Weight",
        "Predicted FEV1",
        "Patient_ID",
    ]
)


def study_duration(x):
    return (x.max() - x.min()).days / 30.5


df["Study duration (months)"] = df.groupby(["ID"])["Date Recorded"].transform(
    lambda x: study_duration(x)
)

df.groupby(["ID"]).agg(["count", "mean", "min", "max"]).style.set_precision(1)

/var/folders/zq/v2r6yn111s3gpdf8lzf72xvw0000gn/T/ipykernel_28750/1313990644.py:12: FutureWarning: ['Date Recorded'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  df.groupby(['ID']).agg(['count', 'mean', 'min', 'max']).style.set_precision(1)
/var/folders/zq/v2r6yn111s3gpdf8lzf72xvw0000gn/T/ipykernel_28750/1313990644.py:12: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  df.groupby(['ID']).agg(['count', 'mean', 'min', 'max']).style.set_precision(1)


In [5]:
df.drop(columns=["ID", "Date Recorded"]).agg(
    ["count", "mean", "min", "max"]
).style.set_precision(1)

/var/folders/zq/v2r6yn111s3gpdf8lzf72xvw0000gn/T/ipykernel_28750/363759384.py:1: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  df.drop(columns=['ID', 'Date Recorded']).agg(['count', 'mean', 'min', 'max']).style.set_precision(1)


,FEV1,Weight (kg),O2 Saturation,Pulse (BPM),Rating,Temp (deg C),Study duration (months)
count,9936.0,13639.0,14011.0,13873.0,39453.0,2034.0,124939.0
mean,1.8,63.8,95.7,85.4,68.1,36.5,6.5
min,0.4,33.0,76.0,41.0,10.0,34.1,0.0
max,5.0,120.8,100.0,157.0,100.0,39.7,20.4


In [16]:
# Bar plot of count of FEV1 measurements per ID using plotly
import plotly.express as px

df_fev1_count = df.groupby(["ID"]).count().reset_index()
# Sort by count of FEV1 measurements
df_fev1_count = df_fev1_count.sort_values(by="FEV1")
fig = px.bar(df_fev1_count, x="ID", y="FEV1")
fig.update_layout(xaxis_tickfont_size=4)
fig.show()

In [15]:
# Bar plot of count of O2 measurements per ID using plotly
import plotly.express as px

df_o2_sat_count = df.groupby(["ID"]).count().reset_index()
# Sort by count of FEV1 measurements
df_o2_sat_count = df_o2_sat_count.sort_values(by="O2 Saturation")
fig = px.bar(df_o2_sat_count, x="ID", y="O2 Saturation")
# Reduce x axis label font size
fig.update_layout(xaxis_tickfont_size=4)
fig.show()